In [8]:
import pandas as pd

# Load the data from the CSV file
file_path = 'Otherdata/bank-additional-full.csv'
data = pd.read_csv(file_path, delimiter=';')  # Assuming delimiter is semicolon based on common format

# Display basic information and the first few rows of the dataset
data_info = data.info()
data_head = data.head()
data_info, data_head
data['cons.price.idx'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

array([93.994, 94.465, 93.918, 93.444, 93.798, 93.2  , 92.756, 92.843,
       93.075, 92.893, 92.963, 92.469, 92.201, 92.379, 92.431, 92.649,
       92.713, 93.369, 93.749, 93.876, 94.055, 94.215, 94.027, 94.199,
       94.601, 94.767])

In [18]:
data_no = data[data['y'] == 'no']
data_yes = data[data['y'] == 'yes']
data_no


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41180,36,admin.,married,university.degree,no,no,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41182,29,unemployed,single,basic.4y,no,yes,no,cellular,nov,fri,...,1,9,1,success,-1.1,94.767,-50.8,1.028,4963.6,no
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no


In [6]:
# def remove_outliers(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
# data = remove_outliers(data, 'age')


In [22]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

# Selecting features for clustering
features = ['age', 'job', 'education', 'marital', 'housing', 'loan', 'cons.price.idx']
data_selected = data_yes[features]

# Define preprocessing for numerical features (scale them)
numerical_features = ['age', 'cons.price.idx']
numerical_transformer = StandardScaler()

# Define preprocessing for categorical features (encode them)
categorical_features = ['job', 'education', 'marital', 'housing', 'loan']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a clustering pipeline
cluster_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('cluster', KMeans(n_clusters=5, random_state=0))])

# Fit the pipeline to the data
cluster_pipeline.fit(data_selected)

# Predict the cluster for each instance
clusters = cluster_pipeline.predict(data_selected)
clusters


array([3, 3, 3, ..., 3, 4, 3])

In [27]:
# Add cluster labels to the original data
data_selected['cluster'] = clusters

# Group by cluster and calculate mean for numerical and mode for categorical attributes
def get_mode(x):
    return x.mode().iloc[0]

def get_median(x):
    return x.median()  

cluster_characteristics = data_selected.groupby('cluster').agg({
    'age': 'mean',
    'job': get_mode,
    'education': get_mode,
    'marital': get_mode,
    'housing': get_mode,
    'loan': get_mode,
    'cons.price.idx': get_mode
}).reset_index()

cluster_characteristics


C:\Users\rbrul\AppData\Local\Temp\ipykernel_38316\95416715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['cluster'] = clusters


,cluster,age,job,education,marital,housing,loan,cons.price.idx
0,0,43.0,admin.,university.degree,married,yes,yes,92.893
1,1,30.0,admin.,university.degree,single,yes,no,92.893
2,2,39.0,admin.,university.degree,married,yes,no,92.893
3,3,35.0,admin.,university.degree,married,yes,no,93.918
4,4,64.0,retired,basic.4y,married,yes,no,93.075
